In [ ]:
from os import environ
# environ["AP_PORT"] = "25432" # str(input('port'))
# README command uses port=25432

In [ ]:
import pickle
import json
import os
import psycopg2
import numpy as np

from spatialyze.database import database
from spatialyze.geospatial_video import GeospatialVideo
from spatialyze.road_network import RoadNetwork
from spatialyze.video_processor.camera_config import camera_config
from spatialyze.video_processor.stages.tracking_3d.tracking_3d import Tracking3DResult
from spatialyze.world import World, _execute
# from spatialyze.video_processor.cache import disable_cache
from spatialyze.video_processor.metadata_json_encoder import MetadataJSONEncoder

In [ ]:
OUTPUT_DIR = '../../data/pipeline/test-results'
VIDEO_DIR =  '../../data/pipeline/videos'
ROAD_DIR = '../../data/scenic/road-network/boston-seaport'

files = os.listdir(VIDEO_DIR)
with open(os.path.join(VIDEO_DIR, 'frames.pkl'), 'rb') as f:
    videos = pickle.load(f)

# disable_cache()

In [ ]:
# database = Database(
#     psycopg2.connect(
#         dbname=environ.get("AP_DB", "mobilitydb"),
#         user=environ.get("AP_USER", "docker"),
#         host=environ.get("AP_HOST", "localhost"),
#         port=environ.get("AP_PORT", "25432"),
#         password=environ.get("AP_PASSWORD", "docker"),
#     )
# )

In [ ]:
world = World(database)
world.addGeogConstructs(RoadNetwork('Boston-Seaport', ROAD_DIR))

In [ ]:
for video in videos.values():
    if video['filename'] not in files:
        continue

    videofile = os.path.join(VIDEO_DIR, video['filename'])
    camera = [camera_config(*c) for c in video['frames']]

    world.addVideo(GeospatialVideo(videofile, camera))

In [ ]:
from spatialyze.utils import F

obj = world.object()
cam = world.camera()
intersection = world.geogConstruct(type='intersection')

world.filter(
    ((obj.type == 'car') | (obj.type == 'truck')) &
    (F.distance(obj, cam) < 50) &
    F.contains(intersection, obj) &
    F.heading_diff(obj, cam, between=[135, 225])
)

In [ ]:
world.getObjects()

In [ ]:
world.saveVideos(outputDir=OUTPUT_DIR, addBoundingBoxes=True)